In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from shutil import which
import time

# from selenium import webdriver

import requests
from lxml.html import fromstring
import json
import os
import re

import pandas as pd

In [39]:
def scroll():
    height = driver.execute_script("return document.body.scrollHeight")
    for scrol in range(100,height,100):
        driver.execute_script(f"window.scrollTo(0,{scrol})")
        time.sleep(0.1)

In [40]:
def extract(string, start, end):

    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [41]:
import urllib.parse as urlparse
def url_fix(s):
    scheme, netloc, path, qs, anchor = urlparse.urlsplit(s)
    path = urlparse.quote(path, '/%')
    qs = urlparse.quote_plus(qs, ':&=')
    url = urlparse.urlunsplit((scheme, netloc, path, qs, anchor))
    img_url = url+'?width=600&format=webp'
    return img_url

In [42]:
len({1,2,1})

2

In [43]:
class louis():
    def __init__(self, items = list(), headless = True):
        
        self.items = items
        self.headless = headless
        
    def scrap(self, category):

        chrome_options = Options()

        if self.headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument("--disable-extensions")


        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)

        # PLease replace your chrome driver path
        chrome_path = r"C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
        global driver
        driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
        driver.get("https://www.louispoulsen.com/en-us/catalog/private")

        time.sleep(1)


        try:
            driver.find_element(By.XPATH, "(//div[@class = 'coi-button-group']/button[@class = 'coi-banner__accept'])[1]").click() 
        except:
            print('cookies request not found')

        main_page = driver.current_url
#         category = '/pendants'
        driver.get(main_page + category)
        url = driver.current_url

        scroll()
        product_url = driver.find_elements(By.XPATH, "//div[@class = 'o-grid']/div")
        items = []
        for product in product_url:
            product_details = {}
            try:
                name_path = product.find_element(By.XPATH, ".//div/article/div/div[2]/div/h2")
                product_details['name'] = name_path.text
            except:
                product_details['name'] = None

#             try:
#                 no_of_heros = product.find_elements(By.XPATH, ".//div/article/div/div/a/div/div/div/div/div/picture/img")
#             except:
#                 no_of_heros = []
#             try: 
#                 if len(no_of_heros) > 0:
#                     hero_path = product.find_element(By.XPATH, ".//div/article/div/div/a/div/div/div/div/div[1]/picture/img")
#                     product_details['hero_image'] = hero_path.get_attribute('src') 

#                 else:
#                     hero_path = product.find_element(By.XPATH, ".//div/article/div/div/a/div/div/picture/img")
#                     product_details['hero_image'] = hero_path.get_attribute('src') 

#             except:
#                 product_details['hero_image'] = None


            product_url_path = product.find_element(By.XPATH, ".//div/article/div/div/a[@href]")
            href = product_url_path.get_attribute("href")
            driver.execute_script("window.open('" +href +"');")
            print(href)
            product_details['product_url'] = href
            # switch to new window
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1)
        #     print(driver.current_url)
            try:
                images = driver.find_elements(By.XPATH, "//div[@class = 'flickity-slider']/div/div/div/picture/img")
                image = []
                for opt in images:
                    image.append(opt.get_attribute("src"))
                display_images = image

            except:
                display_images = []


            try:
                videos_making = driver.find_elements(By.XPATH, "//div[@class = 'u-relative u-w-full u-h-full']/video")
                video = []
                for opt in videos_making:
                    video.append(opt.get_attribute("src"))
                product_details['videos_making'] = '\n'.join(video)

            except:
                product_details['videos_making'] = None

            try:
                videos_animation = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[2]/div[1]/div[3]/div[5]/div[1]/div[1]/div[5]/div[1]/div[1]/div[1]/div[1]/article[1]/div[1]/div[1]/video[1]")
                animations = []
                for opt in videos_animation:
                    animations.append(opt.get_attribute("src"))
                product_details['animations'] = '\n'.join(animations)

            except:
                product_details['animations'] = None


            response = requests.get(href)
            page_source = response.text
            result = extract(page_source, ':data=', '></store-sync>' )

            try:
                product_details['designer'] = extract(result, '"designer":{"name":"', '","url":"/en-us/')
            except:
                product_details['designer'] = None

            try:
                product_details['product description'] = extract(result, '"storyBehind":', '"backgroundColor"')
            except:
                product_details['product description'] = None
                print('product desc not found')



            try:
                technical_description = extract(result, '"featureText":"', '","designer":')
            except:
                technical_description =  ''
#                 print('tech desc not found')
            
            try:
                variants = extract(result, '"variants":', ',"productTracking"' )
                variants = json.loads(variants)
            except:
                print('varients not found')
            
            variant_images = []
            
            colors = []
            sizes = []
            lights = []
            mounts = []
            cables = []
            lightings = []
            
            for var in variants:
                variant_images.append(url_fix(var['images'][0]['url']))
                colors.append(var['colorText'])
                sizes.append(var['sizeText'])
                lights.append(var['lightsourceText'])
                mounts.append(var['mountingText'])
                cables.append(var['cableType'])
                lightings.append(var['features'])
            colors = [i for i in colors if i != "-"]    
            sizes = [i for i in sizes if i != "-"]   
            lights = [i for i in lights if i != "-"]   
            mounts = [i for i in mounts if i != "-"]   
            cables = [i for i in cables if i != "-"]   
            lightings = [i for i in lightings if i != "-"]   
            
            product_details['hero_image'] = variant_images[0]
                
            product_details['images'] = '\n'.join(display_images + variant_images)
            
            variants_num = 0
            
            for varient in variants:
                
                variants_num += 1
                if variants_num >1:
                    product_details['hero_image'] = None
                    
                try:
                    product_details['price'] = varient['rawPrice']

                except:
                    product_details['price'] =  None


                try:
                    product_details['sku'] = varient['id']

                except:
                    product_details['sku'] =  None
                
                if len(variants) >1:
                    
                    
                    try:
                        if len(set(colors))>1:
                            product_details['option(color)'] = varient['colorText']

                    except:
                        product_details['option(color)'] =  None

                    try:
                        if len(set(sizes))>1:
                            product_details['option(size)'] = varient['sizeText']

                    except:
                        product_details['option(size)'] =  None

                    try:
                        if len(set(lights))>1:
                            product_details['option(Light source)'] = varient['lightsourceText']

                    except:
                        product_details['option(Light source)'] =  None

                    try:
                        if len(set(mounts))>1:
                            product_details['option(Mounting type)'] = varient['mountingText']

                    except:
                        product_details['option(Mounting type)'] =  None

                    try:
                        if len(set(cables))>1:
                            product_details['option(Cable type)'] = varient['cableType']

                    except:
                        product_details['option(Cable type)'] =  None

                    try:
                        if len(set(lightings))>1:
                            product_details['option(Lighting control)'] = varient['features']

                    except:
                        product_details['option(Lighting control)'] =  None
                        
                        
                    try:
                        if len(set(switches))>1:
                            product_details['option(Diffusor)'] = varient['shieldDiffusor']
                    except:
                        product_details['option(Diffusor)'] = None


                try:
                    product_details['available_new'] =varient['deliveryTime']

                except:
                    product_details['available_new'] =  None

                try:
                    product_details['lumen'] = varient['lumen']

                except:
                    product_details['lumen'] =  None

                try:
                    product_details['watt'] = varient['watt']

                except:
                    product_details['watt'] =  None

                try:
                    product_details['voltageFrequency'] = varient['voltageFrequency']

                except:
                    product_details['voltageFrequency'] =  None

                try:
                    product_details['kelvin'] = varient['kelvin']

                except:
                    product_details['kelvin'] =  None

                try:
                    product_details['shade'] = varient['shade']

                except:
                    product_details['shade'] =  None

                try:
                    product_details['efficacy'] = varient['efficacy']

                except:
                    product_details['efficacy'] =  None

                try:
                    product_details['reflector'] = varient['reflector']

                except:
                    product_details['reflector'] =  None

                try:
                    product_details['suspension'] = varient['suspension']

                except:
                    product_details['suspension'] =  None

                try:
                    product_details['lightTechnique'] = varient['lightTechnique']

                except:
                    product_details['lightTechnique'] =  None

                try:
                    product_details['additionalFeatures'] = varient['additionalFeatures']

                except:
                    product_details['additionalFeatures'] =  None

                try:
                    product_details['topPlateStyle'] = varient['topPlateStyle']

                except:
                    product_details['topPlateStyle'] =  None

                try:
                    product_details['poleType'] = varient['poleType']

                except:
                    product_details['poleType'] =  None

                try:
                    product_details['transitionToPole'] = varient['transitionToPole']

                except:
                    product_details['transitionToPole'] =  None

                try:
                    product_details['specialAccessories'] = varient['specialAccessories']

                except:
                    product_details['specialAccessories'] =  None

                try:
                    product_details['shieldDiffusor'] = varient['shieldDiffusor']

                except:
                    product_details['shieldDiffusor'] =  None

                try:
                    product_details['body'] = varient['body']

                except:
                    product_details['body'] =  None

                try:
                    product_details['shock'] = varient['shock']

                except:
                    product_details['shock'] =  None

                try:
                    product_details['switchPlug'] = varient['switchPlug']

                except:
                    product_details['switchPlug'] =  None



                product_details['technical description'] = []
                product_details['technical description'].append(technical_description)

                try:
                    technical_description2 = varient['facts'][0]['title']

                except:
                    technical_description2 =  ''
                    print('tech desc 2 not found')
                product_details['technical description'].append(technical_description2)

                try:
                    technical_description3 = varient['facts'][0]['rows'][0]['text']

                except:
                    technical_description3 =  ''
#                     print('tech desc 3 not found') 
                
                product_details['technical description'].append(technical_description3)
                    
                try:
                    technical_description4 = varient['facts'][0]['rows'][1]['text']

                except:
                    technical_description4 =  ''
#                     print('tech desc 4 not found') 
                product_details['technical description'].append(technical_description4)
                
                try:
                    technical_description5 = varient['facts'][1]['title']

                except:
                    technical_description5 =  ''
#                     print('tech desc 5 not found') 
                product_details['technical description'].append(technical_description5)
    
                try:
                    technical_description6 = varient['facts'][1]['rows'][0]['title']

                except:
                    technical_description6 =  ''
#                     print('tech desc 6 not found')  
                product_details['technical description'].append(technical_description6)

                try:
                    technical_description7 = varient['facts'][1]['rows'][0]['text']

                except:
                    technical_description7 =  ''
#                     print('tech desc 7 not found')  

                product_details['technical description'].append(technical_description7)

                try:
                    technical_description8 = varient['facts'][2]['title']

                except:
                    technical_description8 =  ''
#                     print('tech desc 8 not found')
                product_details['technical description'].append(technical_description8)
    
    
                try:
                    technical_description9 = varient['facts'][2]['rows'][0]['text']

                except:
                    technical_description9 =  ''
#                     print('tech desc 9 not found')

                product_details['technical description'].append(technical_description9)
                try:
                    product_details['technical description'] = '\n'.join(product_details['technical description'])
                except:
                    product_details['technical description'] = None
                
                try:   
                    size_weight_data = varient['facts'][0]['rows'][1]['text']
                except:
                    size_weight_data = ''
                    print('size_weight_data not found')

                try:   
                    product_details['width'] = size_weight_data.split('x')[0]
                except:
                    product_details['width'] = None

                try:   
                    product_details['length'] = size_weight_data.split('x')[2].split(',')[0]
                except:
                    product_details['length'] = None

                try:   
                    product_details['height'] = size_weight_data.split('x')[1]
                except:
                    product_details['height'] = None

                try:   
                    product_details['weight'] = size_weight_data.split(',')[-1]
                except:
                    product_details['weight'] = None

                try:   
                    product_details['variant_image'] = url_fix(varient['images'][0]['url'])

                except:
                    product_details['variant_image'] = None
                    print('var img not found')

                try:   
                    product_details['installations'] = url_fix(varient['dimensionImage']['url'])
                except:
                    product_details['installations'] = None
                    print('can img not found')

                try:   
                    product_details['categories'] = url.split('/')[-1]
                except:
                    product_details['categories'] = None
                
    
                self.items.append(product_details.copy()) 
                print(product_details)
            
            driver.close()

            #back to main window
            driver.switch_to.window(driver.window_handles[0])


#         driver.close()
        return self.items


In [65]:
href = 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/skot-wallceiling?v=90655-5743911503-03&t=about&t=history'
response = requests.get(href)
page_source = response.text
result = extract(page_source, ':data=', '></store-sync>' )

In [66]:
variants = extract(result, '"variants":', ',"productTracking"' )
variants = json.loads(variants)
variants

[{'price': '$ 1,204',
  'originalPrice': None,
  'vatText': '',
  'rawPrice': '1204.00',
  'showReadyForShipment': False,
  'id': '90655-5743911422-03',
  'variantCode': '90655-5743911422-03_249767',
  'colorCode': '#adadad',
  'colorText': 'Silver grey',
  'lightsourceText': '15W LED/3000K',
  'mountingText': '-',
  'sizeText': None,
  'armText': None,
  'isDefault': False,
  'deliveryTime': '>8 weeks. Contact Customer Service',
  'availableStatus': 3,
  'images': [{'title': 'LK as Skot Wall Silver grey',
    'url': 'https://louispoulsen.azureedge.net/images/catalog-images/7281_17595_Skot-Wall-Alu-FullOpal-US-01-2-5-90655.png',
    'absoluteUrl': 'https://louispoulsen.azureedge.net/images/catalog-images/7281_17595_Skot-Wall-Alu-FullOpal-US-01-2-5-90655.png',
    'isDefault': True},
   {'title': 'LK as Skot Wall Silver grey',
    'url': 'https://louispoulsen.azureedge.net/images/catalog-images/7281_17595_Skot-Wall-Alu-FullOpal-US-01-2-5-90655.png',
    'absoluteUrl': 'https://louispoul

In [44]:
data_json = []
start = louis(data_json, True)
categories = ['/pendants', '/floor', '/table', '/wall', '/outdoor']
# categories = ['/wall']
for category in categories:
    data = start.scrap(category)

C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_6208/3039636110.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


cookies request not found
https://www.louispoulsen.com/en-us/catalog/private/pendants/ph-3-3-pale-rose-brass-pendant
{'name': 'PH 3½-3 Pale Rose Brass Pendant', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/pendants/ph-3-3-pale-rose-brass-pendant', 'videos_making': '', 'animations': '', 'designer': 'Poul Henningsen', 'product description': '"<p>The System PH was developed to provide efficient, more pleasant and completely glare-free lighting in people’s homes, when electric light began to replace candle chandeliers and kerosene lamps in the late 1920s. Even so, it was commerce and public administration that first took a liking to the highly functional and ultramodern looking PH lamp. Instead, the preferred lamp above the dinner table in most homes remained the chandelier, now electric and equipped with silken shades. Poul Henningsen wanted to show people that with his reflective three-shade system, they could have the cosy atmosphere they were familiar with from th

size_weight_data not found
{'name': 'PH 5 Mini', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/pendants/ph-5-mini', 'videos_making': '', 'animations': '', 'designer': 'Poul Henningsen', 'product description': '"Poul Henningsen developed the PH 5 in 1958 in response to the constant changes made to the shape and size of incandescent bulbs by bulb manufacturers, naming it after the size of the pendant’s main shade, which is 50 centimeters in diameter. When the lamp was introduced, he wrote, typically daringly, about how he had lost faith that the manufacturers of incandescent light bulbs would ever learn to consider common sense or have the consumer’s best interests at heart. “I have accepted fate, and with Louis Poulsen´s permission I have designed a PH fixture, which can be used with any kind of light source, Christmas lights, and 100 W metal filament bulbs. Although a fluorescent tube would be too much to ask in the existing form!” At the time, no one knew that the

{'name': 'PH 3½-3 Pendant', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/pendants/ph-3-3-pendant-us', 'videos_making': '', 'animations': '', 'designer': 'Poul Henningsen', 'product description': '"To mark Poul Henningsen&#39;s 120th birthday on 9 September, Louis Poulsen is launching the new PH 3½-3 pendant. The PH 3½-3 pendant is based on Poul Henningsen&#39;s original drawings from the late1920s and early 1930s, featuring his renowned three-shade system.\\nThe pendant has metal shades, available in green, yellow, red and white. The fixture housing is silk mat brown and made of copper. All colours are based on Poul Henningsen&#39;s colour universe. Like the designer, the fixtures are colourful! As an extra feature, all shades have a white painted rolled edge, accentuating the familiar profile of the three-shade fixture and creating beautiful optical symmetry.\\nThe shades are pressed at Louis Poulsen&#39;s factory in Vejen, by Louis Poulsen&#39;s own staff – a cr

{'name': 'PH Artichoke', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/pendants/ph-artichoke', 'videos_making': '', 'animations': '', 'designer': 'Poul Henningsen', 'product description': '"<p>The PH Artichoke was designed in 1958 by Poul Henningsen for the Langelinie Pavillonen, a modernist Copenhagen restaurant where it continues to enchant visitors today. With its world-famous unique sculptural design, the PH Artichoke is viewed as an international design icon. The fixture has 72 leaves, positioned so as to provide totally glare-free light from any angle. To ensure a high level of quality, much of the production process is still carried out by hand. The renowned light fixture has kept its original copper finish, which to this date illuminates spaces with its warm and ambient lighting. With the brushed and polished steel as well as white painted metal models, further members have been added to the PH Artichoke family, highlighting its versatility and timeless sil

...

{'name': 'Flindt Garden Bollard', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/flindt-garden-bollard', 'videos_making': '', 'animations': '', 'designer': 'Christian Flindt', 'product description': '"The Flindt Garden bollard is the latest addition to the Flindt family, developed by award winning Danish designer and architect Christian Flindt in collaboration with Louis Poulsen. The Flindt Garden bollard is part of Louis Poulsen’s flexible garden collection offering instant outdoor illumination in outdoor living spaces such as private parks, courtyards and gardens without compromising the quality of light or the design. The idea behind the sculptural design, however, was born in 2011, when a more refined type of bollard was desired to illuminate a park next to an exhibition at the Danish art hall Kunsthal Brænderigården in Jutland, today known as Viborg Kunsthal. The fixture was so well received that Louis Poulsen decided to develop it further, together wit

{'name': 'Toldbod 6.1 Wall', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/toldbod-155-wall', 'videos_making': '', 'animations': '', 'designer': None, 'product description': '"Toldbod 6.1\\" Wall was launched in 2006, with a design based on the PH Ellipse reflector. The fixture is very sturdy and discreet. The design is very precise, making the light suitable for a variety of architectural environments.",', 'hero_image': 'https://louispoulsen.azureedge.net/images/catalog-images/7290_17616_Toldbod-155-Wall-Alu-textured-US-01-2-5-91171.png?width=600&format=webp', 'images': 'https://louispoulsen.azureedge.net/4ab10d/globalassets/catalogue-private/outdoor/nyhavn-wall/skagen-04-1.jpg?height=575&format=png\nhttps://louispoulsen.azureedge.net/4ab102/globalassets/catalogue-private/outdoor/nyhavn-wall/skagen-02-1.jpg?height=575&format=png\nhttps://louispoulsen.azureedge.net/4ab116/globalassets/catalogue-private/outdoor/nyhavn-wall/skagen-37-1.jpg?height=575&format=p

{'name': 'PH 3-2½ Wall', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/ph-3-2-wall', 'videos_making': '', 'animations': '', 'designer': 'Poul Henningsen', 'product description': '"Poul Henningsen designed the three-shade system during 1925/1926. The first lights using the system were designed for an exhibition in Paris. His work with Louis Poulsen continued until his death in 1967. Throughout his life, PH sought to create glare-free lighting; aiming to direct light where it was most needed, and creating soft shadows while using incandescent bulbs as a light source. PH 3/2½ Wall is a member of the three-shade family and is one of 19 versions available today. Approximately one thousand different models have been produced over the years and consisted of table, floor and wall lamps, as well as a number of different chandeliers, which were very popular in the 30s for lighting private homes with high ceilings. There were countless combination options. The lights 

{'name': 'AJ Eklipta Outdoor', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/aj-eklipta-3b9c6a57', 'videos_making': '', 'animations': '', 'designer': 'Arne Jacobsen', 'product description': '"AJ Eklipta was designed by Arne Jacobsen in 1959 for Rødovre Town Hall. Two sizes were produced for this project: 13.7\\" and 17.7\\". They were used indoors on stairways and outdoors as wall lamps. A 8.7\\" version was later designed for St. Catherines College in England. Arne Jacobsen preferred the largest model when specifying his own projects. Today, AJ Eklipta is used worldwide and in a variety of architectural contexts.",', 'hero_image': 'https://louispoulsen.azureedge.net/images/catalog-images/7546_17443_AJ-Eklipta-Wall-Outdoor-US-350-01-2-5-91711.png?width=600&format=webp', 'images': 'https://louispoulsen.azureedge.net/4ab50f/globalassets/catalogue-private/outdoor/aj-eklipta/lw_lp_rosenvaengetsalle_0002-1.jpg?height=575&format=png\nhttps://louispoulsen.azureedg

{'name': 'Skot Wall', 'product_url': 'https://www.louispoulsen.com/en-us/catalog/private/outdoor/skot-wallceiling', 'videos_making': '', 'animations': '', 'designer': None, 'product description': '"The Skot light was developed in the early 1900s by LK (Lauritz Knudsen). It was originally developed for ships and maritime areas, but it has also become popular in other environments for its compact and sturdy design.",', 'hero_image': 'https://louispoulsen.azureedge.net/images/catalog-images/7281_17595_Skot-Wall-Alu-FullOpal-US-01-2-5-90655.png?width=600&format=webp', 'images': 'https://louispoulsen.azureedge.net/images/catalog-images/7281_17595_Skot-Wall-Alu-FullOpal-US-01-2-5-90655.png?width=600&format=webp\nhttps://louispoulsen.azureedge.net/images/catalog-images/7282_17596_Skot-Wall-Alu-HalfOpal-US-02-2-5-90655.png?width=600&format=webp\nhttps://louispoulsen.azureedge.net/images/catalog-images/10792_17608_Skot-Wall-Alu-FullOpal-01-2-5-90655.png?width=600&format=webp\nhttps://louispouls

In [45]:
len(data)

600

In [46]:
len(data_json)

600

In [47]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data_json, )
df


,name,product_url,videos_making,animations,designer,product description,hero_image,images,price,sku,...,weight,variant_image,installations,categories,option(color),option(Light source),option(Cable type),option(size),option(Lighting control),option(Mounting type)
0,PH 3½-3 Pale Rose Brass Pendant,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""<p>The System PH was developed to provide eff...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/49c7ac/glob...,2072.00,91861-5741939794-03,...,5.5 lbs,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,pendants,NaN,NaN,NaN,NaN,NaN,NaN
1,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741940149-03,...,,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,pendants,Pastels blue rose peach,A-21/medium,White textile,NaN,NaN,NaN
2,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918032-03,...,,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,pendants,Hues of rose,1x22W A-21/medium,White textile,NaN,NaN,NaN
3,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918045-03,...,,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,pendants,Hues of grey,1x22W A-21/medium,White textile,NaN,NaN,NaN
4,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918058-03,...,,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,pendants,Hues of orange,1x22W A-21/medium,White textile,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,AJ Eklipta Outdoor,https://www.louispoulsen.com/en-us/catalog/pri...,,,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",None,https://louispoulsen.azureedge.net/4ab50f/glob...,1038.00,91711-5842905426-03,...,9.0 lbs,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,outdoor,NaN,1x100W A-19/IF medium,NaN,17.7 IN,NaN,NaN
596,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911422-03,...,5.9 lbs,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,outdoor,Silver grey,NaN,NaN,NaN,NaN,NaN
597,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911435-03,...,6.5 lbs,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,outdoor,Silver grey,NaN,NaN,NaN,NaN,NaN
598,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911503-03,...,5.9 lbs,https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,outdoor,Natural paint aluminum,NaN,NaN,NaN,NaN,NaN


In [48]:
len(df.name.unique())

87

In [49]:
df['product description'] = df['product description'].apply(lambda d: d.replace('<p>', ''))
df['product description'] = df['product description'].apply(lambda d: d.replace('</p>', ''))
df['product description'] = df['product description'].apply(lambda d: d[:-1])


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      600 non-null    object
 1   product_url               600 non-null    object
 2   videos_making             600 non-null    object
 3   animations                600 non-null    object
 4   designer                  564 non-null    object
 5   product description       600 non-null    object
 6   hero_image                89 non-null     object
 7   images                    600 non-null    object
 8   price                     598 non-null    object
 9   sku                       600 non-null    object
 10  available_new             600 non-null    object
 11  lumen                     357 non-null    object
 12  watt                      357 non-null    object
 13  voltageFrequency          541 non-null    object
 14  kelvin                    

In [51]:
df['color'] = df['option(color)']

In [52]:
df = df.replace("-", None)

In [53]:
df_full = df.reindex(columns=[*df.columns.tolist(), 'GTIN', 'option(finish)', 'finish','display_dimensions', 'depth', \
                    'diameter', 'arm height', 'seat height', 'table clearance height', \
                    'maximum dimensions', 'files','made_in', 'canopy dimensions',\
                              'collection' , 'bulb_type', 'bulb_text', 'UL Listed', \
                    'certifications', 'warranty & care', 'position', 'notes'], fill_value= None)

In [54]:
df_full

,name,product_url,videos_making,animations,designer,product description,hero_image,images,price,sku,...,made_in,canopy dimensions,collection,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,PH 3½-3 Pale Rose Brass Pendant,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""The System PH was developed to provide effici...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/49c7ac/glob...,2072.00,91861-5741939794-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741940149-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918032-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918045-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918058-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,AJ Eklipta Outdoor,https://www.louispoulsen.com/en-us/catalog/pri...,,,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",None,https://louispoulsen.azureedge.net/4ab50f/glob...,1038.00,91711-5842905426-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911422-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911435-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911503-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_full = df_full.drop_duplicates(subset='sku', keep='first', ignore_index=True)
df_full

,name,product_url,videos_making,animations,designer,product description,hero_image,images,price,sku,...,made_in,canopy dimensions,collection,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,PH 3½-3 Pale Rose Brass Pendant,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""The System PH was developed to provide effici...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/49c7ac/glob...,2072.00,91861-5741939794-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741940149-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918032-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918045-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918058-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,AJ Eklipta Outdoor,https://www.louispoulsen.com/en-us/catalog/pri...,,,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",None,https://louispoulsen.azureedge.net/4ab50f/glob...,1038.00,91711-5842905426-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911422-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
573,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911435-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911503-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_full.to_csv('louispoulsen_new.csv')

In [4]:
df_full = pd.read_csv('louispoulsen_new.csv')
df_full

,Unnamed: 0,name,hero_image,product_url,videos_making,animations,designer,product description,images,price,...,canopy dimensions,collection,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,0,PH 3½-3 Pale Rose Brass Pendant,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Poul Henningsen,"""The System PH was developed to provide effici...",https://louispoulsen.azureedge.net/49c7ac/glob...,2072.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/4a24b6/glob...,1224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/4a24b6/glob...,1224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/4a24b6/glob...,1224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/4a24b6/glob...,1224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,565,AJ Eklipta Outdoor,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",https://louispoulsen.azureedge.net/4ab50f/glob...,1038.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,566,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,1204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,567,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,1204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568,568,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,1204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_tech = pd.read_excel('louispoulsen.xlsx')
df_tech

,Unnamed: 0,name,hero_image,product_url,images,videos_making,animations,designer,product description,price,...,canopy dimensions,collection,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,0,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4a24b6/glob...,https://player.vimeo.com/external/429972165.hd...,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...","$ 1,162",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4a24b6/glob...,https://player.vimeo.com/external/429972165.hd...,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...","$ 1,162",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4a24b6/glob...,https://player.vimeo.com/external/429972165.hd...,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...","$ 1,162",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4a24b6/glob...,https://player.vimeo.com/external/429972165.hd...,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...","$ 1,162",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,PH 5,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4a24b6/glob...,https://player.vimeo.com/external/429972165.hd...,NaN,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...","$ 1,162",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,AJ Eklipta Outdoor,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,https://louispoulsen.azureedge.net/4ab50f/glob...,NaN,NaN,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",$ 990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622,622,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,NaN,"""The Skot light was developed in the early 190...","$ 1,146",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,623,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,NaN,"""The Skot light was developed in the early 190...","$ 1,146",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,624,Skot Wall,https://louispoulsen.azureedge.net/images/cata...,https://www.louispoulsen.com/en-us/catalog/pri...,NaN,NaN,NaN,NaN,"""The Skot light was developed in the early 190...","$ 1,146",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df_tech = df_tech[['sku', 'technical description']]
df_tech = df_tech.rename(columns={'technical description': 'technical_description_new'})
df_tech

,sku,technical_description_new
0,90293-5741918113-03,The fixture provides 100% glare-free light. It...
1,90293-5741918032-03,The fixture provides 100% glare-free light. It...
2,90293-5741918045-03,The fixture provides 100% glare-free light. It...
3,90293-5741918058-03,The fixture provides 100% glare-free light. It...
4,90293-5741918061-03,The fixture provides 100% glare-free light. It...
...,...,...
621,91711-5842905426-03,The fixture emits soft comfortable light. The ...
622,90655-5743911422-03,The fixture is available with two types of dis...
623,90655-5743911435-03,The fixture is available with two types of dis...
624,90655-5743911503-03,The fixture is available with two types of dis...


In [58]:
df_data = pd.merge(df_full, df_tech,  how ='left', on = ['sku'])
df_data

,name,product_url,videos_making,animations,designer,product description,hero_image,images,price,sku,...,canopy dimensions,collection,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes,technical_description_new
0,PH 3½-3 Pale Rose Brass Pendant,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""The System PH was developed to provide effici...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/49c7ac/glob...,2072.00,91861-5741939794-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741940149-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918032-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
3,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918045-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
4,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918058-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,AJ Eklipta Outdoor,https://www.louispoulsen.com/en-us/catalog/pri...,,,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",None,https://louispoulsen.azureedge.net/4ab50f/glob...,1038.00,91711-5842905426-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture emits soft comfortable light. The ...
626,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911422-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...
627,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911435-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...
628,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911503-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...


In [59]:
df_data = df_data.drop_duplicates(subset='sku', keep='first', ignore_index=True)
df_data

,name,product_url,videos_making,animations,designer,product description,hero_image,images,price,sku,...,canopy dimensions,collection,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes,technical_description_new
0,PH 3½-3 Pale Rose Brass Pendant,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""The System PH was developed to provide effici...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/49c7ac/glob...,2072.00,91861-5741939794-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741940149-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918032-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
3,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918045-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
4,PH 5,https://www.louispoulsen.com/en-us/catalog/pri...,,,Poul Henningsen,"""Poul Henningsen developed the PH 5 in 1958 in...",None,https://louispoulsen.azureedge.net/4a24b6/glob...,1224.00,90293-5741918058-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture provides 100% glare-free light. It...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,AJ Eklipta Outdoor,https://www.louispoulsen.com/en-us/catalog/pri...,,,Arne Jacobsen,"""AJ Eklipta was designed by Arne Jacobsen in 1...",None,https://louispoulsen.azureedge.net/4ab50f/glob...,1038.00,91711-5842905426-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture emits soft comfortable light. The ...
572,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",https://louispoulsen.azureedge.net/images/cata...,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911422-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...
573,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911435-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...
574,Skot Wall,https://www.louispoulsen.com/en-us/catalog/pri...,,,None,"""The Skot light was developed in the early 190...",None,https://louispoulsen.azureedge.net/images/cata...,1204.00,90655-5743911503-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The fixture is available with two types of dis...


In [60]:
import numpy as np
df_data = df_data.replace({np.nan: None})

In [61]:
for i in range(len(df_data)):
    if df_data['technical description'][i] is None:
        df_data['technical description'][i] = df_data['technical_description_new'][i]

In [62]:
df_data['technical description'].isnull().sum()

4

In [63]:
len(df_data['name'].unique())

87

In [64]:
df_data.to_csv('lp.csv')

In [ ]:
df_data

In [18]:
k = []
k.extend([1,2])
k

[1, 2]